4.24 修改意见

再拓展一下，把全部2天的数据都算了
测试5的点位有没有找出来？
我猜大部分时间花在了IO上面。你看看能不能再这之前把要计算的通道选好，内存读到数据后做计算，计算1个通道的psd跟100个通道应该差不太多，这样吧，把每20个通道算一个结果，我们把这条路上的信号水平都算出来
不要直接画图。我们需要中间过程的结果来做更深入的分析，把中间结果保存好


In [ ]:
# last modified 2024/4/24 
# %matplotlib ipympl
%matplotlib inline
import h5py 
import glob
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from DasTools import DasPrep as dp
import os
import datetime
import matplotlib
import logging
import csv
import time
def concat(flist , start = 0 , span = 20 ):
    datalist = []
    for fname in flist[ start : start + span ]:
        data = dp.read_das(fname)[::10]
        datalist.append(data)

    data = np.concatenate(datalist, axis = 1)
    utc_datetime = datetime.datetime.strptime(flist[start][-22 : -7],'%Y%m%d_%H%M%S') + datetime.timedelta(hours=+8)
    utc_day = utc_datetime.strftime("%m-%d %H:%M")
    return data , utc_datetime, utc_day 


def psd(h5file , startn,endn):
    with h5py.File(h5file, 'a') as f:
    # 创建一个dataset
        span_each_pic = 20 #12h -> 50min
        concat_time = 0
        ch_list = [10*i for i in range(148)]
        for n in range(startn, endn):
            data , utc_datetime, utc_day = concat(flist , start= span_each_pic * n ,span= span_each_pic)
            nfft = 30000
            fs = 1000
            start = time.time()
            for i in range(148):
                ch = ch_list[i]
                if not f.__contains__(str(ch_list[i])):   #判断这个组存不存在
                    g = f.create_group(str(ch_list[i]))
                else:
                    g = f[str(ch_list[i])]
                # g =  f.create_group(str(ch_list[i]))  
                [Pxx1,f1] = plt.psd(data[i],                   # 随机信号
                                NFFT=nfft,               # 每个窗的长度
                                Fs=fs,                   # 采样频率   
                                detrend='mean',          # 去掉均值
                                window=np.hanning(nfft), # 加汉尼窗
                                noverlap=int(nfft*3/4),  # 每个窗重叠75%的数据
                                sides='onesided')        # 求单边谱
                plt.close
                # print(f1)
                g.create_dataset(str(utc_day),  data=Pxx1)
            concat_time +=  time.time() -start
            print(n , "拼接运行时间:%.2f min %d s"%( int(concat_time / 60) , concat_time%60) , datetime.datetime.now() , utc_day, "check in ")
        f.close()

flist = glob.glob('../../czyc/test3/*357.h5')
flist.sort()
print(len(flist))

path = '../output/DAS/'
if not os.path.exists(path):
    os.makedirs(path)

print(int(len(flist)/20))

# h5fname = '../output/DAS/psd_4_25_1.hdf5'
# psd(h5fname , 0 , 72 ) #31m39s / 14min 18s --->6h
# h5fname = '../output/DAS/psd_4_25_2.hdf5'
# psd(h5fname , 136 , 144 ) #31m39s / 14min 18s --->



In [ ]:
# h5fname = '../output/DAS/psd_4_25_3_.hdf5'
# psd(h5fname , 144 , 216 ) #27m35s / 11min 33s --->

In [ ]:
h5fname = '../output/DAS/psd_4_25_4_.hdf5'
psd(h5fname , 216 , 288 ) #31m39s / 14min 18s --->

In [ ]:
# h5fname = '../output/DAS/psd_4_25_5.hdf5'
# psd(h5fname , 288 , 360 ) #36m57s / 13min 20s --->

In [ ]:
# h5fname = '../output/DAS/psd_4_25_6.hdf5'
# psd(h5fname , 360 , 432 ) #34m29s / 13min 54s --->

In [ ]:
h5fname = '../output/DAS/psd_4_25_7.hdf5'
psd(h5fname , 432 , 433 ) #31m39s / 14min 18s --->

In [87]:
import h5py 
import numpy as np
import matplotlib.pyplot as plt
# ch_list = [40,150,300,450,600,700,830,910,1000,1200,1420]
ch_list = range(0,1479,10)
for ch in ch_list:
    psd = []
    all_time = []
    for fi in range(1,8):
        with h5py.File('../output/DAS/psd_4_25_'+str(fi)+'.hdf5', 'r') as f:
            if len(f[str(ch)].keys()) == 1 :
                dset = f[str(ch)]['01']
            else:
                dset = f[str(ch)]
            time = [key for key in dset.keys()]
            # print(time)
            # start = time.index('20:00:12')
            # time = time[start:]+time[:start]
            # print(time)

            psd.append(  [dset[key][:] for key in time])
            all_time.append(time[:])
            f.close()
    print("-----------read",ch,"------------")
    
    print(len(psd))
    print(len(all_time))
    psd = np.concatenate(psd)
    time = np.concatenate(all_time)
    print(psd.shape)
    print(len(time))
    psd = np.log(psd)
    plt.figure(figsize=(30,10))
    plt.imshow(psd.T, aspect='auto', cmap='jet',vmin=0, vmax=12)
    # plt.imshow(psd.T, aspect='auto', cmap='viridis')
    plt.grid(alpha = 1)

    xstick = range(0, len(time) , 20)
    ystick = [1500*i for i in range(10)]
    # print(f['frequency'].shape)

    plt.xticks(xstick , ["1."+time[i][-7]+" "+time[i][-5:]  for i in xstick],rotation = 0)
    # plt.xticks(xstick , [time[i][:2]+" "+time[i][3:]  for i in xstick],rotation = 0)

    plt.yticks(ystick , ['%d'% (i/30.)  for i in ystick])
    plt.ylabel("Frequency")
    plt.xlabel("Time")
    plt.title("log(PSD) Date=1.3-1.5 Channel="+str(ch)+" nfft=30,000")

    plt.colorbar()
    plt.savefig('../output/DAS/ch_'+str(ch)+'_psd_.png') #10s per pic
    plt.close()


-----------read 0 ------------
7
7
(482, 15001)
482
-----------read 10 ------------
7
7
(482, 15001)
482
-----------read 20 ------------
7
7
(482, 15001)
482
-----------read 30 ------------
7
7
(482, 15001)
482
-----------read 40 ------------
7
7
(482, 15001)
482
-----------read 50 ------------
7
7
(482, 15001)
482
-----------read 60 ------------
7
7
(482, 15001)
482
-----------read 70 ------------
7
7
(482, 15001)
482
-----------read 80 ------------
7
7
(482, 15001)
482
-----------read 90 ------------
7
7
(482, 15001)
482
-----------read 100 ------------
7
7
(482, 15001)
482
-----------read 110 ------------
7
7
(482, 15001)
482
-----------read 120 ------------
7
7
(482, 15001)
482
-----------read 130 ------------
7
7
(482, 15001)
482
-----------read 140 ------------
7
7
(482, 15001)
482
-----------read 150 ------------
7
7
(482, 15001)
482
-----------read 160 ------------
7
7
(482, 15001)
482
-----------read 170 ------------
7
7
(482, 15001)
482
-----------read 180 ------------
7
7
(4

4.26 我们现在可以取噪音录音的时间，把所有通道的结果画出来，画一个图，横轴是通道，比如选取下午2-3点的psd，每个通道作为一个列

In [ ]:
import h5py 
import numpy as np
import matplotlib.pyplot as plt
all_psd = []
ch_list = range(0,1479,10)
for ch in ch_list:
    psd = []
    for fi in range(1,8):
        with h5py.File('../output/DAS/psd_4_25_'+str(fi)+'.hdf5', 'r') as f:
            if len(f[str(ch)].keys()) == 1 :
                dset = f[str(ch)]['01']
            else:
                dset = f[str(ch)]
            time = [key for key in dset.keys()]
            print("time",fi,time)
            start = time.index('04 15:10') #fi = 3  
            end= time.index('04 15:40')
            # start = time.index('01-05 03:10') #fi = 5  test5: 3:10-3:40
            # end= time.index('01-05 03:40')
            time = time[start:end+1]
            # print(time)

            psd.append(  [dset[key][:] for key in time])
            all_time.append(time[:])
            f.close()
    print("-----------read",ch,"------------")
    
    print("len(psd)",len(psd))
    psd = np.mean(psd,axis=1) 
    print("len(psd)",len(psd))
    print("psd.shape",psd.shape)
    print("len(time)",len(time))
    psd = np.log(psd)
    all_psd.append(psd[0])
    print("len all_psd",len(all_psd))
all_psd= np.array(all_psd)
print(all_psd.shape)
plt.figure(figsize=(30,8))
plt.imshow(all_psd.T, aspect='auto', cmap='jet',vmin=0, vmax=12)
# plt.imshow(psd.T, aspect='auto', cmap='viridis')

xstick = range(0, len(ch_list) , 5 )
ystick = [1500*i for i in range(10)]
# print(f['frequency'].shape)

plt.xticks(xstick ,[ch_list[i]  for i in xstick] ,rotation = 0)

plt.yticks(ystick , ['%d'% (i/30.)  for i in ystick])
plt.ylabel("Frequency")
plt.xlabel("Channel")
plt.grid()

# plt.title("log(mean(PSD)) test1=1.4 14:10-14:40 nfft=30,000")
# plt.title("log(mean(PSD)) test2=1.4 14:40-15:10 nfft=30,000")
plt.title("log(mean(PSD)) test3=1.4 15:10-15:40 nfft=30,000")
# plt.title("log(mean(PSD)) test4=1.5 02:00-02:25 nfft=30,000")
# plt.title("log(mean(PSD)) test5=1.5  03:10-03:40 nfft=30,000")

plt.colorbar()
plt.savefig('../output/DAS/test3_psd.png') #10s per pic
plt.show()
plt.close()
#5-10s
